<a href="https://colab.research.google.com/github/koppls/EURO2020_assignment/blob/main/UEFA_Sports_ResearchPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6 MB 32 kB/s 


In [ ]:
# for mathematics operations
import numpy as np
import pandas as pd

# for optimization
from pulp import *
import pandas as pd

In [ ]:
df = pd.read_csv('player_data_qualifying_cleaned_v6.txt', encoding='utf-8')

In [ ]:
df['attempts_off_target'].replace(np.nan, 0, inplace = True) 
df[165:171]

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides
165,Denmark,Ronnow,Goalkeeper,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0
166,Denmark,Skov,Midfielder,3,0,242,3,0,0,2,91.7,131,118,66.7,4,2,1,2,0,0,0,2,1,1,14,5,5.0,0,0,4,2,2,0
167,Denmark,Eriksen,Midfielder,5,0,720,8,0,0,4,82.0,525,430,50.5,72,38,31,4,5,0,0,0,0,0,0,0,8.0,0,0,0,4,48,2
168,Denmark,J. Hansen,Goalkeeper,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0
169,Denmark,Durmisi,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0
170,Denmark,Wass,Midfielder,0,0,208,3,0,0,3,85.7,203,177,46.7,11,3,4,4,1,0,0,6,0,2,2,0,2.0,0,0,0,2,0,0


In [ ]:
# df.drop(['fouls_suffered'], axis=1, inplace = True)
# df.drop(df.loc[(df['country'] != 'Denmark') and (df['country'] != 'England') and (df['country'] != 'Italy')].index, inplace=True)
df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0


In [ ]:
# we chose to use the top 2 quartiles of each attribue to determine the score per attribute 
for col in list(df.columns)[5:]:
  top10 = df[col].quantile(q = 0.9) # top 10%
  top20 = df[col].quantile(q = 0.8) # top 20%
  top30 = df[col].quantile(q = 0.7) # top 30%
    
  if top10 != 0:
    # attribute values in top 10% receive score of +3
    df.loc[df[col] >= top10, f"{col}_score"] = 3
    # attribute values in top 20% receive score of +2
    df.loc[(df[col] >= top20) & (df[col] < top10), f"{col}_score"] = 2
    # attribute values in top 30% receive score of +1
    df.loc[(df[col] >= top30) & (df[col] < top20), f"{col}_score"] = 1
    # rest receives score of 0 (below 30%)
    df.loc[df[col] < top30, f"{col}_score"] = 0
  else:
    df.loc[df[col] >= top10, f"{col}_score"] = df[col]

top10_gk_ballsRecovered = df['balls_recovered'][df['position'] == 'Goalkeeper'].quantile(q = 0.9)
df.loc[df['balls_recovered'] >= top10_gk_ballsRecovered, ["balls_recovered_score"]] = 3
top20_gk_ballsRecovered = df['balls_recovered'][df['position'] == 'Goalkeeper'].quantile(q = 0.8)
df.loc[(df['balls_recovered'] >= top20_gk_ballsRecovered) & (df['balls_recovered'] < top10_gk_ballsRecovered), ["balls_recovered_score"]] = 2
top30_gk_ballsRecovered = df['balls_recovered'][df['position'] == 'Goalkeeper'].quantile(q = 0.7)
df.loc[(df['balls_recovered'] >= top30_gk_ballsRecovered) & (df['balls_recovered'] < top20_gk_ballsRecovered), ["balls_recovered_score"]] = 1
df.loc[df['balls_recovered'] < top30_gk_ballsRecovered, ["balls_recovered_score"]] = 0
top10_field_ballsRecovered = df['balls_recovered.1'][df['position'] != 'Goalkeeper'].quantile(q = 0.9)
df.loc[df['balls_recovered.1'] >= top10_field_ballsRecovered, ["balls_recovered.1_score"]] = 3
top20_field_ballsRecovered = df['balls_recovered.1'][df['position'] != 'Goalkeeper'].quantile(q = 0.8)
df.loc[(df['balls_recovered.1'] >= top20_field_ballsRecovered) & (df['balls_recovered'] < top10_field_ballsRecovered), ["balls_recovered.1_score"]] = 2
top30_field_ballsRecovered = df['balls_recovered.1'][df['position'] != 'Goalkeeper'].quantile(q = 0.7)
df.loc[(df['balls_recovered.1'] >= top30_field_ballsRecovered) & (df['balls_recovered'] < top20_field_ballsRecovered), ["balls_recovered.1_score"]] = 1
df.loc[df['balls_recovered.1'] < top30_field_ballsRecovered, ["balls_recovered.1_score"]] = 0

In [ ]:
# yellow and red cards reduce score
df['fouls_committed_score'] = df['fouls_committed_score'] *-1 
df['yellow_cards_score'] = df['yellow_cards_score'] *-1
df['red_cards_score'] = df['red_cards_score']* -1
df['attempts_off_target_score'] = df['attempts_off_target_score'] *-0.5
df['offsides_score'] = df['offsides_score'] *-0.5

df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_suffered_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-2.0,0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0,1.0,3.0,3.0,-0.5
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2,3.0,3.0,0,0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,-2.0,-3.0,0,1.0,1.0,0.0,3.0,3.0,-1.5,0,0,3.0,3.0,3.0,-1.5
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13,3.0,3.0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,-3.0,-3.0,0,0.0,2.0,0.0,3.0,3.0,-1.5,0,1,3.0,3.0,2.0,-1.5
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5


In [ ]:
# compute Player Score - add all attribute scores (columns) for each player and store in new column
df['playerScore'] = df[list(df)[df.columns.get_loc('offsides') + 1:]].sum(axis = 1)
df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_suffered_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score,playerScore
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-2.0,0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0,1.0,3.0,3.0,-0.5,4.0
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2,3.0,3.0,0,0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,-2.0,-3.0,0,1.0,1.0,0.0,3.0,3.0,-1.5,0,0,3.0,3.0,3.0,-1.5,39.0
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13,3.0,3.0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,-3.0,-3.0,0,0.0,2.0,0.0,3.0,3.0,-1.5,0,1,3.0,3.0,2.0,-1.5,22.0
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5


In [ ]:
df['playerScore'] = df['playerScore'] + df['goals']
df['playerScore'] = df['playerScore'] - df['goals_conceded']
df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_suffered_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score,playerScore
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-2.0,0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0,1.0,3.0,3.0,-0.5,4.0
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2,3.0,3.0,0,0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,-2.0,-3.0,0,1.0,1.0,0.0,3.0,3.0,-1.5,0,0,3.0,3.0,3.0,-1.5,41.0
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13,3.0,3.0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,-3.0,-3.0,0,0.0,2.0,0.0,3.0,3.0,-1.5,0,1,3.0,3.0,2.0,-1.5,28.0
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5


In [ ]:
df['country_rank_score'] = df['country']
df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_suffered_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score,playerScore,country_rank_score
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5,Austria
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-2.0,0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0,1.0,3.0,3.0,-0.5,4.0,Austria
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2,3.0,3.0,0,0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,-2.0,-3.0,0,1.0,1.0,0.0,3.0,3.0,-1.5,0,0,3.0,3.0,3.0,-1.5,41.0,Austria
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13,3.0,3.0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,-3.0,-3.0,0,0.0,2.0,0.0,3.0,3.0,-1.5,0,1,3.0,3.0,2.0,-1.5,28.0,Austria
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5,Austria


In [ ]:
# mapping country to ranking score from qualifying rounds
rankScoringMap = {
            'Belgium' : 24.0,
            'Italy' : 22.0,
            'Spain' : 20.0,
            'France' : 18.0,
            'Poland' : 16.0,
            'Russia' : 14.0,
            'Turkey' : 12.0,
            'England' : 10.0,
            'Germany' : 9.0,
            'Sweden' : 8.0,
            'Ukraine' : 7.0,
            'NorthMacedonia' : 6.0,
            'Netherlands' : 5.0,
            'Austria' : 4.0,
            'Finland' : 3.0,
            'Hungary' : 2.0,
            'Portugal' : 1.0,
            'Switzerland' : 1.0,
            'Croatia' : 1.0,
            'Scotland' : 1.0,
            'Slovakia' : 1.0,
            'Denmark' : 1.0,
            'CzechRepublic' : 1.0,
            'Wales' : 1.0,
}

df['country_rank_score'] = df['country_rank_score'].replace(rankScoringMap)
df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_suffered_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score,playerScore,country_rank_score
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5,4.0
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-2.0,0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0,1.0,3.0,3.0,-0.5,4.0,4.0
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2,3.0,3.0,0,0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,-2.0,-3.0,0,1.0,1.0,0.0,3.0,3.0,-1.5,0,0,3.0,3.0,3.0,-1.5,41.0,4.0
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13,3.0,3.0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,-3.0,-3.0,0,0.0,2.0,0.0,3.0,3.0,-1.5,0,1,3.0,3.0,2.0,-1.5,28.0,4.0
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,1.5,4.0


In [ ]:
df['playerScore'] = df['playerScore'] + df['country_rank_score']
df.head()

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_suffered,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_suffered_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score,playerScore,country_rank_score
0,Austria,Lienhart,Defender,0,0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,5.5,4.0
1,Austria,Zulj,Midfielder,0,0,85,1,0,0,0,0.0,0,0,0.0,0,0,0,1,2,1,0,1,0,0,3,0,2.0,0,0,1,1,4,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-2.0,0,0.0,0.0,0.0,0.0,0.0,-0.5,0,0,1.0,3.0,3.0,-0.5,8.0,4.0
2,Austria,Lazaro,Midfielder,2,0,754,9,0,0,5,83.6,407,340,58.1,64,37,13,10,6,2,0,16,1,3,19,5,11.0,0,0,3,1,37,2,3.0,3.0,0,0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,-2.0,-3.0,0,1.0,1.0,0.0,3.0,3.0,-1.5,0,0,3.0,3.0,3.0,-1.5,45.0,4.0
3,Austria,Arnautovic,Forward,6,0,710,8,0,0,3,81.1,242,197,18.8,5,2,3,5,17,2,0,10,2,0,37,18,11.0,0,1,8,2,0,13,3.0,3.0,0,0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,-3.0,-3.0,0,0.0,2.0,0.0,3.0,3.0,-1.5,0,1,3.0,3.0,2.0,-1.5,32.0,4.0
4,Austria,Ranftl,Midfielder,0,0,13,1,0,0,1,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0,0.0,0.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,5.5,4.0


In [ ]:
df.loc[df['names'] == "Shaw"]

NameError: ignored

In [ ]:
# define budget
budget = 105000000

In [ ]:
# positions for traditional 4-4-2 formation
positions = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
countries = ['Italy', 'England', 'Denmark']

In [ ]:
# generate data structures for maximization problem
players = list(df['names'])
# salaries = dict(zip(players, df['price']))
positions = dict(zip(players, df['position']))
scores = dict(zip(players, df['playerScore'])) # newly created player score
countries = dict(zip(players, df['country']))

# Set Players to Take either 1 or 0 values (owned or not)
player_vars = LpVariable.dicts("names", players, lowBound=0, upBound=1, cat='Integer')

In [ ]:
total_score = LpProblem("TeamScore", LpMaximize)

In [ ]:
total_score += lpSum([scores[i] * player_vars[i] for i in player_vars])

In [ ]:
# add team budget constraint 
# total_score += lpSum([salaries[i] * player_vars[i] for i in player_vars]) <= budget

In [ ]:
# Get indices of players for each position 
gk = [p for p in positions.keys() if positions[p] == 'Goalkeeper']
d = [p for p in positions.keys() if positions[p] == 'Defender']
mid = [p for p in positions.keys() if positions[p] == 'Midfielder']
fwd = [p for p in positions.keys() if positions[p] == 'Forward']

bel = [c for c in countries.keys() if countries[c] == 'Belgium']
ita = [c for c in countries.keys() if countries[c] == 'Italy']
esp = [c for c in countries.keys() if countries[c] == 'Spain']
fra = [c for c in countries.keys() if countries[c] == 'France']
pol = [c for c in countries.keys() if countries[c] == 'Poland']
rus = [c for c in countries.keys() if countries[c] == 'Russia']
tur = [c for c in countries.keys() if countries[c] == 'Turkey']
eng = [c for c in countries.keys() if countries[c] == 'England']
ger = [c for c in countries.keys() if countries[c] == 'Germany']
swe = [c for c in countries.keys() if countries[c] == 'Sweden']
ukr = [c for c in countries.keys() if countries[c] == 'Ukraine']
nma = [c for c in countries.keys() if countries[c] == 'NorthMacedonia']
nel = [c for c in countries.keys() if countries[c] == 'Netherlands']
aut = [c for c in countries.keys() if countries[c] == 'Austria']
fin = [c for c in countries.keys() if countries[c] == 'Finland']
hun = [c for c in countries.keys() if countries[c] == 'Hungary']
por = [c for c in countries.keys() if countries[c] == 'Portugal']
swi = [c for c in countries.keys() if countries[c] == 'Switzerland']
cro = [c for c in countries.keys() if countries[c] == 'Croatia']
sco = [c for c in countries.keys() if countries[c] == 'Scotland']
slo = [c for c in countries.keys() if countries[c] == 'Slovakia']
den = [c for c in countries.keys() if countries[c] == 'Denmark']
cze = [c for c in countries.keys() if countries[c] == 'CzechRepublic']
wal = [c for c in countries.keys() if countries[c] == 'Wales']


# Set Constraints
# you can only select 1 goalkeeper, 4 defenders, 4 midfielders and 2 forwards
total_score += lpSum([player_vars[i] for i in gk]) == 1
total_score += lpSum([player_vars[i] for i in d]) == 4
total_score += lpSum([player_vars[i] for i in mid]) == 4
total_score += lpSum([player_vars[i] for i in fwd]) == 2

# you can only select 3 players from each country
total_score += lpSum([player_vars[i] for i in bel]) <= 0
total_score += lpSum([player_vars[i] for i in ita]) <= 6
total_score += lpSum([player_vars[i] for i in esp]) <= 0
total_score += lpSum([player_vars[i] for i in fra]) <= 0
total_score += lpSum([player_vars[i] for i in pol]) <= 0
total_score += lpSum([player_vars[i] for i in rus]) <= 0
total_score += lpSum([player_vars[i] for i in tur]) <= 0
total_score += lpSum([player_vars[i] for i in eng]) <= 6
total_score += lpSum([player_vars[i] for i in ger]) <= 0
total_score += lpSum([player_vars[i] for i in swe]) <= 0
total_score += lpSum([player_vars[i] for i in ukr]) <= 0
total_score += lpSum([player_vars[i] for i in nma]) <= 0
total_score += lpSum([player_vars[i] for i in nel]) <= 0
total_score += lpSum([player_vars[i] for i in aut]) <= 0
total_score += lpSum([player_vars[i] for i in fin]) <= 0
total_score += lpSum([player_vars[i] for i in hun]) <= 0
total_score += lpSum([player_vars[i] for i in por]) <= 0
total_score += lpSum([player_vars[i] for i in swi]) <= 0
total_score += lpSum([player_vars[i] for i in cro]) <= 0
total_score += lpSum([player_vars[i] for i in sco]) <= 0
total_score += lpSum([player_vars[i] for i in slo]) <= 0
total_score += lpSum([player_vars[i] for i in den]) <= 6
total_score += lpSum([player_vars[i] for i in cze]) <= 0
total_score += lpSum([player_vars[i] for i in wal]) <= 0


In [ ]:
# Solve team score maximization problem
total_score.solve()

1

In [ ]:
# Team Composition
df_final = pd.DataFrame()

for v in total_score.variables():
  if v.varValue > 0:
     df_final = df_final.append(df.query(f"names == '{v.name.split('names_')[1].replace('_', ' ')}'"))

df_final

,country,names,position,goals,goals_conceded,minutes_played,matches_played,balls_recovered,saves_from_penalties,clean_sheets,passing_accuracy,passes_attempted,passes_completed,crossing_accuracy,crosses_attempted,crosses_completed,freekicks_taken,fouls_committed,yellow_cards,red_cards,balls_recovered.1,tackles,clearances_attempted,total_attempts,attempts_on_target,attempts_off_target,attempts_crossbar,attempts_post,attempts_blocked,assists,corners_taken,offsides,minutes_played_score,matches_played_score,balls_recovered_score,saves_from_penalties_score,clean_sheets_score,passing_accuracy_score,passes_attempted_score,passes_completed_score,crossing_accuracy_score,crosses_attempted_score,crosses_completed_score,freekicks_taken_score,fouls_committed_score,yellow_cards_score,red_cards_score,balls_recovered.1_score,tackles_score,clearances_attempted_score,total_attempts_score,attempts_on_target_score,attempts_off_target_score,attempts_crossbar_score,attempts_post_score,attempts_blocked_score,assists_score,corners_taken_score,offsides_score,playerScore,country_rank_score
367,Italy,Barella,Midfielder,3,0,651,8,0,0,5,85.0,314,267,43.8,8,5,4,12,1,0,15,3,6,11,6,3.0,0,0,2,2,0,1,2.0,3.0,0,0,3.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0,-3.0,-2.0,0,1.0,3.0,1.0,2.0,3.0,-1.0,0,0,2.0,3.0,2.0,-1.0,53.0,22.0
200,England,Barkley,Midfielder,4,0,417,6,0,0,3,82.7,340,296,45.0,19,7,14,2,1,0,8,1,2,9,4,2.0,0,0,3,2,14,0,1.0,1.0,0,0,1.0,0.0,2.0,2.0,2.0,3.0,2.0,3.0,-0.0,-2.0,0,0.0,1.0,0.0,2.0,2.0,-0.5,0,0,3.0,3.0,3.0,-0.5,42.0,10.0
359,Italy,Bonucci,Defender,1,0,780,10,0,0,7,89.1,732,654,0.0,2,0,8,4,1,0,46,1,22,8,1,4.0,0,0,3,3,0,0,3.0,3.0,0,0,3.0,2.0,3.0,3.0,0.0,0.0,0.0,2.0,-1.0,-2.0,0,1.0,1.0,3.0,2.0,1.0,-1.0,0,0,3.0,3.0,2.0,-0.5,53.5,22.0
184,England,Chilwell,Defender,0,0,450,5,0,0,4,86.0,361,311,42.0,24,12,12,3,0,0,20,0,11,1,1,0.0,0,0,0,3,10,0,1.0,1.0,0,0,2.0,0.0,2.0,2.0,2.0,3.0,3.0,3.0,-0.0,-1.0,0,1.0,0.0,2.0,0.0,1.0,-0.0,0,0,0.0,3.0,3.0,-0.5,37.5,10.0
372,Italy,Emerson,Defender,0,0,346,5,0,0,3,88.4,221,190,19.8,26,8,6,1,0,0,14,4,5,1,0,1.0,0,0,0,2,5,1,0.0,1.0,0,0,1.0,1.0,1.0,1.0,0.0,3.0,2.0,2.0,-0.0,-1.0,0,1.0,3.0,1.0,0.0,0.0,-0.0,0,0,0.0,3.0,3.0,-1.0,43.0,22.0
381,Italy,Jorginho,Midfielder,3,0,777,9,0,0,5,91.2,898,822,11.1,3,2,15,9,2,0,38,3,7,8,4,2.0,0,0,2,1,0,0,3.0,3.0,0,0,3.0,2.0,3.0,3.0,0.0,0.0,1.0,3.0,-3.0,-3.0,0,1.0,3.0,2.0,2.0,2.0,-0.5,0,0,2.0,3.0,2.0,-0.5,56.0,22.0
195,England,Kane,Forward,12,0,667,8,0,0,5,78.8,175,135,27.1,8,4,1,5,0,0,6,1,6,30,20,3.0,0,2,7,5,1,2,2.0,3.0,0,0,3.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,-2.0,-1.0,0,0.0,1.0,1.0,3.0,3.0,-1.0,0,2,3.0,3.0,2.0,-1.5,46.5,10.0
182,England,Keane,Defender,1,0,450,5,0,0,2,92.6,386,360,0.0,1,0,3,4,1,0,19,0,14,5,1,2.0,0,0,2,1,0,0,1.0,1.0,0,0,0.0,3.0,3.0,3.0,0.0,0.0,0.0,1.0,-1.0,-2.0,0,1.0,0.0,3.0,1.0,1.0,-0.5,0,0,2.0,3.0,2.0,-0.5,32.0,10.0
393,Italy,Sirigu,Goalkeeper,0,1,437,5,13,0,4,94.6,105,99,0.0,0,0,7,0,0,0,0,1,3,0,0,0.0,0,0,0,0,0,0,1.0,1.0,2,0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,-0.0,-1.0,0,0.0,1.0,0.0,0.0,0.0,-0.0,0,0,0.0,1.0,2.0,-0.5,34.5,22.0
176,England,Sterling,Forward,8,0,574,7,0,0,4,83.6,262,218,35.7,13,3,0,12,1,0,7,0,2,17,13,2.0,0,0,2,7,0,6,2.0,2.0,0,0,2.0,0.0,1.0,1.0,2.0,2.0,1.0,0.0,-3.0,-2.0,0,0.0,0.0,0.0,3.0,3.0,-0.5,0,0,2.0,3.0,2.0,-1.5,37.0,10.0
